# Install/ Import Packages

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier 

In [ ]:
!pip install autogluon

In [ ]:
!pip install explainerdashboard

In [3]:
train = pd.read_csv("/content/drive/MyDrive/2023hilton/train_to_colab.csv")
test = pd.read_csv("/content/drive/MyDrive/2023hilton/test_to_colab.csv")

## **AutoGluon:**

In [21]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='IntentToStayHighLow',eval_metric='roc_auc').fit(train, time_limit=120)  # Fit models for 120s
leaderboard = predictor.leaderboard(test)
leaderboard

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.922339,0.918589,0.013804,0.005935,0.620970,0.013804,0.005935,0.620970,1,True,4
1,CatBoost,0.921797,0.922068,0.006185,0.002098,2.114687,0.006185,0.002098,2.114687,1,True,7
2,RandomForestEntr,0.921180,0.919795,0.220653,0.108498,3.237057,0.220653,0.108498,3.237057,1,True,6
3,WeightedEnsemble_L2,0.920897,0.923807,1.665918,0.705425,27.153208,0.008512,0.000595,0.422776,2,True,13
4,RandomForestGini,0.920732,0.920049,0.218925,0.110815,1.844725,0.218925,0.110815,1.844725,1,True,5
5,ExtraTreesGini,0.920101,0.922388,0.256835,0.117181,1.590409,0.256835,0.117181,1.590409,1,True,8
6,ExtraTreesEntr,0.919134,0.920677,0.242000,0.128715,1.502596,0.242000,0.128715,1.502596,1,True,9
7,LightGBMLarge,0.917567,0.916775,0.034848,0.007576,1.138383,0.034848,0.007576,1.138383,1,True,12
8,NeuralNetFastAI,0.914275,0.915897,0.063888,0.024296,7.452350,0.063888,0.024296,7.452350,1,True,10
9,LightGBMXT,0.912800,0.915319,0.026820,0.007933,0.846233,0.026820,0.007933,0.846233,1,True,3


In [22]:
from sklearn.metrics import *
roc_auc_score(test.IntentToStayHighLow,predictor.predict_proba(test)[1])

0.9208965643052599

For testing custom models (e.g., the ones that you fine-tuned), you can refer to https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-custom-model.html

# **XGBoost and Explainer Dashboard**

In [15]:
clf = XGBClassifier(objective = "binary:logistic", random_state = 1)
clf.fit(train.drop(columns = "IntentToStayHighLow"), train.IntentToStayHighLow)

XGBClassifier(random_state=1)

In [19]:
roc_auc_score(test.IntentToStayHighLow,clf.predict_proba(test.drop(columns = "IntentToStayHighLow"))[:,1])

0.9217311036789296

In [ ]:
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
explainer = ClassifierExplainer(clf, test.drop(columns = "IntentToStayHighLow"), test.IntentToStayHighLow)

Detected XGBClassifier model: Changing class type to XGBClassifierExplainer...
Note: model_output=='probability'. For XGBClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...


In [ ]:
ExplainerDashboard(explainer, mode='inline').run(port=8051)

Building ExplainerDashboard..
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
Generating layout...
Calculating shap values...
Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Generating xgboost model dump...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating predictions...
Calculating pred_percentiles...
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard inline (terminate it with ExplainerDashboard.terminate(8051))


<IPython.core.display.Javascript object>